In [ ]:
pip install fuzzywuzzy

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
members = pd.read_csv('./data/cleaned_members.csv')

In [3]:
main = pd.read_csv('./data/senate_and_house.csv')

In [4]:
members.head()

,people,state,party,total_senate_years,total_house_years,total_years
0,James Abdnor,South Dakota,Republican,6,8,14
1,Neil Abercrombie,Hawaii,Democratic,0,22,22
2,James Abourezk,South Dakota,Democratic,6,2,8
3,"Ralph Abraham, Lee",Louisiana,Republican,0,6,6
4,Spencer Abraham,Michigan,Republican,6,0,6


In [5]:
main.head()

,Unnamed: 0,transaction_date,ticker,asset_description,type,amount,representative,timestamp
0,4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,1584504000
1,5,2020-03-24,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585022400
2,6,2020-03-31,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585627200
3,10,2020-02-21,ENV,"Envestnet, Inc",sale_partial,"$1,001 - $15,000",Susie Lee,1582261200
4,12,2020-03-17,LKQ,LKQ Corporation,sale_partial,"$1,001 - $15,000",Susie Lee,1584417600


In [6]:
# unique in main
names_in_main = main.representative.unique()

# made into a list 
names = []
for name in names_in_main:
    names.append(name)
type(names)

#list of members df 
member_names = []
for name in members.people.values:
    member_names.append(name)

In [7]:
len(member_names)

2422

In [8]:
# unique names in main
keys = {}

for name in names:
    #finding closest math to main data from larger 'members data'
    keys[name] = ((process.extract(name, member_names, limit=2)))

In [9]:
keys

{'Susie Lee': [('Susie Lee', 100), ('Ralph Abraham, Lee', 86)],
 'Alan Lowenthal': [('Alan Lowenthal, S.', 95), ('Allan Howe', 67)],
 'Brooks': [('Jack Brooks, B.', 90), ('Mo Brooks', 90)],
 'Robert Wittman': [('Robert Wittman, J.', 95),
  ('Robert Kastenmeier, W.', 86)],
 'Lois Frankel': [('Lois Frankel', 100), ('Al Franken', 73)],
 'Michael McCaul': [('Michael McCaul, T.', 95),
  ('Michael Fitzpatrick, G.', 86)],
 'Suzan DelBene': [('Suzan DelBene, K.', 95), ('la de', 72)],
 'Greg Gianforte': [('Greg Gianforte', 100), ('Greg Ganske', 72)],
 'Lloyd Smucker': [('Lloyd Smucker', 100), ('William Scott, Lloyd', 86)],
 'Earl Blumenauer': [('Earl Blumenauer', 100), ('Earl Hutto', 86)],
 'James Comer': [('James Comer', 100), ('James Broyhill, T.', 86)],
 'James Langevin': [('James Langevin, R.', 95), ('James Longley', 74)],
 'Anthony Gonzalez': [('Anthony Gonzalez', 100),
  ('Gonzalez-Colon, Commissioner, Jenniffer', 86)],
 'Josh Gottheimer': [('Josh Gottheimer', 100), ('Joshua Eilberg', 62)

In [10]:
#where scores live
# for i in range(1, 173):
#     print(keys[i][1][1])

In [11]:
merge_key = [keys[name][0][0] for name in main['representative']]

In [12]:
main['merge_key'] = merge_key

In [13]:
main = main.merge(members, how = 'left', left_on = 'merge_key', right_on='people')

In [14]:
main.to_csv('./data/megan_senate_house_merged_years_served.csv', index=False)

# Merge Donations with Main

In [15]:
donations = pd.read_csv('data/congress_donations.csv')
donations['party'] = [name[-2] for name in donations['Name']]
donations.columns = donations.columns.str.lower().str.replace(' ','_')
donations['name'] = [name[:-4] for name in donations['name']]

In [16]:
df = pd.read_csv('./data/megan_senate_house_merged_years_served.csv')

In [17]:
df.head()

,Unnamed: 0,transaction_date,ticker,asset_description,type,amount,representative,timestamp,merge_key,people,state,party,total_senate_years,total_house_years,total_years
0,4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,1584504000,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
1,5,2020-03-24,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585022400,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
2,6,2020-03-31,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585627200,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
3,10,2020-02-21,ENV,"Envestnet, Inc",sale_partial,"$1,001 - $15,000",Susie Lee,1582261200,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
4,12,2020-03-17,LKQ,LKQ Corporation,sale_partial,"$1,001 - $15,000",Susie Lee,1584417600,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2


In [18]:
donations.head()

,name,total_raised,total_from_small_donors,percent_from_small_donors*,total_money_raised,party
0,Ocasio-CortezAlexandria Ocasio-Cortez,$20664795,$16434594,79.53%,Raised over $100k,D
1,TrahanLori Trahan,$1449303,$1151465,79.45%,Raised over $100k,D
2,SandersBernie Sanders,$21565004,$15151822,70.26%,Raised over $100k,D
3,HunterDuncan D. Hunter,$1693623,$1142274,67.45%,Raised over $100k,R
4,JordanJim Jordan,$18313823,$12122206,66.19%,Raised over $100k,R


In [19]:
def clean_name(name):
    first = name.split()[0]
    second = name.split()[1:]
    first_rev = first[::-1]
    for i, let in enumerate(first_rev):
        if let == let.upper():
            index = i
            break
    for word in second:
        if len(word) < 3 or '\.' in word:
            second.remove(word)
    first = first[-(i+1):]
    second.insert(0,first)
    new = ' '.join(second)
    return new

In [20]:
donations['name'] = [clean_name(name) for name in donations['name']]

In [21]:
len(donations.name.unique())

537

In [22]:
donations.shape

(537, 6)

In [23]:
len(df.merge_key.unique())

169

In [24]:
df.columns

Index(['Unnamed: 0', 'transaction_date', 'ticker', 'asset_description', 'type',
       'amount', 'representative', 'timestamp', 'merge_key', 'people', 'state',
       'party', 'total_senate_years', 'total_house_years', 'total_years'],
      dtype='object')

In [25]:
# get unique list of names in main
names_in_df = df.representative.unique()
# make into a list 
names = []
for name in names_in_df:
    names.append(name)
type(names)
#list of members df 
member_names = []
for name in donations.name.values:
    member_names.append(name)

In [26]:
keys = {}
for name in names:  #names in smaller dataset to use as merge key
    #finding closest math to main data from larger 'members data'
    keys[name] = ((process.extract(name, member_names, limit=2))) 
    #you can limit to 1 to take less time but I wanted to check

In [27]:
for name in keys:
    if keys[name][0][1] <= 86:
        print(name, keys[name])

James Langevin [('Jim Langevin', 85), ('Mike Levin', 67)]
Gerald Connolly [('Gerry Connolly', 83), ('J Cox', 54)]
Kathy Manning [('Kay Granger', 58), ('Kathy Castor', 56)]
Michael Simpson [('Mike Simpson', 81), ('Michael Burgess', 67)]
Rohit Khanna [('Ro Khanna', 86), ('Lori Trahan', 52)]
Christopher Jacobs [('Chris Jacobs', 86), ('Christopher Murphy', 72)]
eter Meijer [('Peter DeFazio', 58), ('Peter Welch', 55)]
James Hon Banks [('Jim Banks', 86), ('James Lankford', 66)]
Neal Patrick MD, Facs Dunn [('Patrick Leahy', 86), ('Neal Dunn', 86)]
Sara Jacobs [('Chris Jacobs', 70), ('Jose Serrano', 58)]
Marie Newman [('Madeleine Dean', 62), ('Mike Conaway', 58)]
Gilbert Cisneros [('Gil Cisneros', 86), ('Bernie Sanders', 60)]
Michael Conaway [('Mike Conaway', 81), ('Michael Waltz', 71)]
Neal Patrick MD, FACS Dunn [('Patrick Leahy', 86), ('Neal Dunn', 86)]
James Banks [('Jim Banks', 80), ('James Lankford', 72)]
Donald Sternoff Beyer [('Don Beyer', 86), ('Jon Tester', 63)]
Michael Garcia [('Mike

In [28]:
no_match = ['Kathy Manning', 'Michael Enzi', 'Michael Enzi', 'John Hickenlooper', 'William Hagerty',
           'Robert "Bobby" Scott', 'cott Franklin', 'Victoria Spartz', 'David Cheston Rouzer',
           'Marjorie Taylor Greene', 'Marie Newman', 'eter Meijer', 'Kathy Manning']

second_match = ['Mitchell Mcconnell, Jr.', 'Harold Dallas Rogers', 'Neal Patrick Dunn MD, FACS']

In [29]:
keys[name][1][0]

'Cory Gardner'

In [30]:
donation_merge_key = [keys[name][1][0] if name in second_match \
                      else None if name in no_match \
                      else keys[name][0][0] for name in df['representative']]

In [31]:
donation_merge_key

['Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Mo Brooks',
 'Mo Brooks',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Rob Wittman',
 'Rob Wittman',
 'Rob Wittman',
 'Rob Wittman',
 'Rob Wittman',
 'Lois Frankel',
 'Lois Frankel',
 'Lois Frankel',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',

In [32]:
df['donation_merge_key'] = donation_merge_key

In [33]:
df.head()

,Unnamed: 0,transaction_date,ticker,asset_description,type,amount,representative,timestamp,merge_key,people,state,party,total_senate_years,total_house_years,total_years,donation_merge_key
0,4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,1584504000,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee
1,5,2020-03-24,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585022400,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee
2,6,2020-03-31,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585627200,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee
3,10,2020-02-21,ENV,"Envestnet, Inc",sale_partial,"$1,001 - $15,000",Susie Lee,1582261200,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee
4,12,2020-03-17,LKQ,LKQ Corporation,sale_partial,"$1,001 - $15,000",Susie Lee,1584417600,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee


In [34]:
df = df.merge(donations, how = 'left', left_on = 'donation_merge_key', right_on='name')

In [36]:
df.columns

Index(['Unnamed: 0', 'transaction_date', 'ticker', 'asset_description', 'type',
       'amount', 'representative', 'timestamp', 'merge_key', 'people', 'state',
       'party_x', 'total_senate_years', 'total_house_years', 'total_years',
       'donation_merge_key', 'name', 'total_raised', 'total_from_small_donors',
       'percent_from_small_donors*', 'total_money_raised', 'party_y'],
      dtype='object')

In [37]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [38]:
df.columns

Index(['transaction_date', 'ticker', 'asset_description', 'type', 'amount',
       'representative', 'timestamp', 'merge_key', 'people', 'state',
       'party_x', 'total_senate_years', 'total_house_years', 'total_years',
       'donation_merge_key', 'name', 'total_raised', 'total_from_small_donors',
       'percent_from_small_donors*', 'total_money_raised', 'party_y'],
      dtype='object')

In [39]:
df.to_csv('./house_senate_years_donations.csv', index = False)